# **The N-body problem**

In physics, the N-body problem is used to compute interactions between particles. You can found this problem in many fields, like gravitational interaction among the stars in a galaxy or the Coulomb forces exerted by the atoms in a molecule.

The N-body problem can be formulated as

$$
U(x_i)=\sum_jF(x_i,x_j)
$$

where $x_i$ are the positions of the particles, $U(x_i)$ is a physical quantity at $x_i$ and $F(x_i, x_j)$ represents the interaction between particles $i$ and $j$.

In the following, we will be interested to simulate a galaxy with several bodies (stars, dusts, planets, ...). These bodies are represented by their position, their velocity and their mass. We use the newton's second law which says that mass times acceleration is equal to the sum of the forces on the mass. And the forces are computed using the Newton's law of universal gravitation and given by this formula

$$
F_{ij} = \frac{Gm_im_j(x_j-x_i)}{\left|x_j-x_i\right|^3},
$$

where $m_i$ is the mass and $x_i$ the position of the particle $i$ and $\left|x_j-x_i\right|$ is the distance between particles $i$ and $j$.

The acceleration is then given by this formula

$$
m_i\frac{d^2x_i}{dt^2}=\sum_{j\neq i}F_{ij}.
$$

As you can see, we have to solve an ODE. There are several iterative methods to solve numerically this equation. But before to choose one of theses schemes, let's take a look of the computation of the forces.

Suppose that we have $n$ bodies. A naive algortihm to compute the forces is

In [ ]:
for i in range(n):
    for j in range(n):
        if i != j:
            F[i] += G*m[i]*m[j]*(x[j]-x[i])/distance(x[i], x[j])**3

This algorithm is of complexity $n^2$ !! 

It is a really slow algorithm. It is possible to improve the computation of the interactions between particles by using the Barnes-Hut algorithm. The idea is to divide recursively the $n$ bodies into group by storing them in a quad-tree for 2D problem and to say that if a group of bodies is sufficiently far away from an other bodies, we can approximate its graviationnal effects by using its center of mass. The center of mass of a group of bodies is the average position of a body in that group, weighted by mass. This algorithm is of complexity $nlog(n)$.

## **Building the tree**

Let's take the example given on this excellent post (http://arborjs.org/docs/barnes-hut). We encourage you to read carefully this article to deeper understand the algorithm.

![example space](./figures/example-space.png)

The generated tree is the following

![example tree](./figures/example-tree.png)

The algorithm for the construction of the tree is to insert the bodies one after anatoher. We use a recursive procedure to insert a body $b$ into the tree at node $x$.

   1. if the node $x$ is empty, put the new body $b$ here,
   2. if the node $x$ is a quadrant, recall the procedure for the body $b$ in this quadrant,
   3. if the node $x$ is a body $c$, subdivide the region until bodies $b$ and $c$ are in diefferent quadrant. Then, insert bodies $b$ and $c$ in the right quadrants.
    
In order to use Cython, Pythran or Numba, it is not suitable to use a tree structure in Python and it is preferable to store the tree as an array. We can do that by specifying the tree as follow

![tree array](./figures/tree_array.png)

In our example, the construction of the tree will give us the array

![example tree array](./figures/example-tree-array.png)

where $0,1, \dots, 7$ represent the bodies $A, B, \dots, H$, $-1$ means that there is no body on this quadrant. Cell index is represented by an integer greater that $n-1$ where $n$ is the number of bodies.

To move to the cell $k$ in this array, we just have to compute the following index $n + 4(k+n)$.

## **Calculating the mass and the center of mass**

Now that we have construct our tree, we can calculate the center of mass and the total mass of the cell using this algorithm

   1. initialize the total_mass and the center_of_mass arrays with size $n_{bodies} + n_{cell}$
   2. store the mass and the coordinates of the bodies at the beginning of the arrays total_mass and center_of_mass
   3. loop over the cells starting by the end
       1. find the elements of the cell
       2. sum the mass of each element which are bodies or cells
       3. compute the center_of_mass which is the sum of the coordinates divide by their mass of each element
       4. normalize the center_of_mass by its total_mass
       
The previous algorithm can be written in Python by

In [ ]:
import numpy as np

total_mass = np.zeros(nbodies+ncells)
center_of_mass = np.zeros((nbodies+ncells, 2))

for i in range(nbodies):
    total_mass[i] = mass_of_body[i]
    center_of_mass[i] = coordinates_of_body[i]
    
for c in range(ncell, -1, -1):
    elements = tree[nbodies + 4*c:nbodies + 4*c + 4]
    total_mass[nbodies + c] = np.sum(total_mass[elements[elements>=0]])
    center_of_mass[nbodies + i] = np.sum(center_of_mass[elements[elements>=0]]
                                         *total_mass[elements[elements>=0], np.newaxis]
                                         , axis=0)
    center_of_mass[nbodies + i] /= total_mass[nbodies + i]    

## **Calculating the forces**

To calculate the net force acting on body b, use the following recursive procedure, starting with the root of the quad-tree

   1. If the current node is an external node (and it is not body b), calculate the force exerted by the current node on b using the Newton's law of universal gravitation, and add this amount to b’s net force.
   2. Otherwise, calculate the ratio $\frac{s}{d}$. If $\frac{s}{d} < \theta$, treat this internal node as a single body, and calculate the force it exerts on body b by using the center_of_mass for the position and the total_mass for the mass using again the Newton's law of universal gravitation, and add this amount to b’s net force.
   3. Otherwise, run the procedure recursively on each of the current node’s children.

## **Solve the ODE**

Now that we can compute the forces of our system, we can solve the ODE by using an iterative method. Suppose that you want to solve the following ODE

$$
y'(t) = f(t,y).
$$

The Adam Bashforth of order $6$ solves numerically this equation by this formula

$$
y_{k+1} = y_k + \Delta t \left(a_0f(t_{k}, y_{k})+a_1f(t_{k-1}, y_{k-1}) + a_2f(t_{k-2}, y_{k-2})+a_3f(t_{k-3}, y_{k-3})+a_4f(t_{k-4}, y_{k-4})+a_5f(t_{k-5}, y_{k-5})\right)
$$

where

$$
\begin{array}{l}
a_0 = \frac{4277}{1440}, \\
a_1 = \frac{-7923}{1440}, \\
a_2 = \frac{ 9982}{1440}, \\
a_3 = \frac{-7298}{1440}, \\
a_4 = \frac{2877}{1440}, \\
a_5 = \frac{-475}{1440}. 
\end{array}
$$

To initialize this scheme, we will use a Runge Kutta method of order $4$ to calculate the first solutions.

In the N-body problem, the unknowns are the coordinates and the velocities of the system. Calculating the forces gives us the acceleration of the system. With this acceleration, we can calculate the velocities at the time step $k+1$. The new positions at time step $k+1$ are calculated using the velocities at time step $k, k-1, \dots, k-5$.

In [1]:
# execute this part to modify the css style
from IPython.core.display import HTML
def css_styling():
    styles = open("./style/custom.css", "r").read()
    return HTML(styles)
css_styling()